In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import os
import sys
import findspark

In [4]:
spark_path = '/Users/screative/.sdkman/candidates/spark/current/'

In [5]:
os.environ['SPARK_HOME'] = spark_path
os.environ['HADOOP_HOME'] = spark_path

In [6]:
# !pip install py4j

In [7]:
sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/pyhton/pyspark")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.7-src.zip")

In [8]:
from pyspark import SparkContext

In [9]:
from pyspark import SparkConf

In [10]:
sc = SparkContext("local[4]", "test app")

In [11]:
sc
#sc.stop()

<SparkContext master=local[4] appName=test app>

In [12]:
sc.stop()

### Resilient Distributed Data (RDD) Operations

In [13]:
sc = SparkContext(master='local[4]', appName='Sticky data')

In [14]:
sc

<SparkContext master=local[4] appName=Sticky data>

In [ ]:
data = [12,32,45, 65, 89]

In [ ]:
rdd1 = sc.parallelize(data)

In [ ]:
print(rdd1)

In [ ]:
rdd1.collect()

In [ ]:
rdd1.count()

In [ ]:
sc.version

In [ ]:
sc.pythonVer

In [ ]:
sc.master

In [ ]:
sc.appName

In [ ]:
rdd1.getNumPartitions()

In [ ]:
sc.parallelize([]).isEmpty()

In [ ]:
sc.parallelize(data).isEmpty()

In [ ]:
# Basic Stats
rdd1.collect()

In [ ]:
rdd1.max()

In [ ]:
rdd1.min()

In [ ]:
rdd1.mean()

In [ ]:
rdd1.stdev()

In [ ]:
rdd1.variance()

In [ ]:
rdd1.stats()

In [ ]:
from pyspark.mllib.stat import Statistics

In [ ]:
matx = sc.parallelize(
    [np.array([10.1,12.4,14.5,16.8,21.3]),
    np.array([21.3, 24.2, 35.4, 36.4, 31.7]),
    np.array([21.1, 23.0, 54.0, 65.1,71.0])]
)

In [ ]:
summary = Statistics.colStats(matx)

In [ ]:
summary.mean()

In [ ]:
summary.variance()

In [ ]:
summary.numNonzeros()

In [ ]:
# Correlation
X = sc.parallelize([10.1,12.4,14.5,16.8,21.3])
Y = sc.parallelize([21.3, 24.2, 35.4, 36.4, 31.7])

In [ ]:
corr = Statistics.corr(X,Y, method='pearson')

In [ ]:
corr

In [ ]:
from pyspark.mllib.linalg import Matrices, Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.stat import Statistics

In [ ]:
vec = Vectors.dense(10.1,12.4,14.5,16.8,21.3,21.3, 24.2, 35.4, 36.4, 31.7)

In [ ]:
vec

In [ ]:
goodnessTest = Statistics.chiSqTest(vec)

In [ ]:
print(goodnessTest)

In [ ]:
# Logistic Regression Model
# Wine Quality dataset
# https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv
data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data", header=None, sep=',')

In [ ]:
data.head()

In [ ]:
# names = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality.names")
sc.stop()

In [ ]:
# from pyspark.sql import SparkSession

# spark = SparkSession \
#     .builder \
#     .appName("Models-Eque") \
#     .master("local[4]") \
#     .getOrCreate()

In [15]:
from pyspark.mllib.regression import LabeledPoint, LinearRegressionModel, LinearRegressionWithSGD

In [17]:
data_source = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data"

In [ ]:
# !wget "https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data" wine.csv

In [18]:
data = sc.textFile("wine.data") \
        .map(lambda line: line.split(",")) \
        .filter(lambda line: len(line) > 1) \
        .map(lambda line: (line[0], line[3], line[2])) \
        .collect()
    

In [21]:
data[:4]

[('1', '2.43', '1.71'),
 ('1', '2.14', '1.78'),
 ('1', '2.67', '2.36'),
 ('1', '2.5', '1.95')]

In [ ]:
[10.1,12.4,14.5,16.8,21.3]
[21.3, 24.2, 35.4, 36.4, 31.7]
[21.1, 23.0, 54.0, 65.1,71.0]

In [33]:
parsed_data = [LabeledPoint(0.0, [10.1,12.4,14.5,16.8,21.3]),
               LabeledPoint(0.0, [21.3, 24.2, 35.4, 36.4, 31.7]),
               LabeledPoint(1.0, [24.1, 23.0, 54.0, 65.1,71.0]),
               LabeledPoint(1.0, [11.1, 23.0, 54.0, 65.1,71.0]),
               LabeledPoint(2.0, [50.1,12.4,14.5,16.8,21.3]),
               LabeledPoint(2.0, [31.3, 24.2, 35.4, 36.4, 31.7])              
              ]

In [34]:
from pyspark.mllib.classification import LogisticRegressionModel, LogisticRegressionWithLBFGS, LogisticRegressionWithSGD

In [35]:
model = LogisticRegressionWithLBFGS.train(sc.parallelize(parsed_data), numClasses=3)

In [36]:
model.intercept

0.0

In [37]:
model.weights

DenseVector([-2.0936, -5.1068, 1.1102, 1.1913, 1.303, 4.0311, -1.8149, -0.3459, -0.532, -1.0338])